In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import random
import os
import math

#use GPU running
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
#generate mobility data for input with array
data1 = np.load('5min_1node_data.npy')
data100 = np.load('5min_100node_data_id.npy')

print(len(data1))
print(len(data100))
print(data1[0])
print(data100[0])

16800
28000
[5209.9303 9194.304 ]
[   0.     5242.8832 9167.2133]


In [6]:
data1_len = len(data1)
data100_len = len(data100)
print(len(data1))
print(len(data100))

x = []
y = []
for i in range(0, data100_len-500):
  x.append([[data100[j][1],data100[j][2]] for j in range(i, i+500, 100)])
  y.append([data100[i+500][1],data100[i+500][2]])
print(len(x))
print(len(y))

16800
28000
27500
27500


In [7]:
x = torch.tensor(x, dtype= torch.float32)
y = torch.tensor(y, dtype = torch.float32)
print(x[150])
print(y[150])
print(x.size())
print(y.size())

tensor([[5713.8970, 2061.4768],
        [5713.8970, 2061.4768],
        [5713.8970, 2061.4768],
        [5713.8970, 2061.4768],
        [5713.8970, 2061.4768]])
tensor([5709.4053, 2065.0637])
torch.Size([27500, 5, 2])
torch.Size([27500, 2])


In [8]:
#initialization
batch_size = 15
#num_epochs = 10
#input_size = 108
#hidden_size = 216
#layer_num = 3

#input data to dataset
class myselfDataset(Dataset):
  def __init__(self, x, y):
    self.x  = x
    self.y  = y
    self.len = len(x)
  def __getitem__(self,index):
    return self.x[index], self.y[index]
  def __len__(self):
    return self.len

#random distribution with train and test set
def get_dataloader(batch_size):
  dataset = myselfDataset(x,y)
  train_len = int(0.9 * len(dataset))
  length = [train_len, len(dataset)-train_len]
  trainset, validset = random_split(dataset,length)
  #train_x = x[:train_len]
  #train_y = y[:train_len]
  #valid_x = x[train_len:]
  #valid_y = y[train_len:]
  #trainset = myselfDataset(train_x, train_y)
  #validset = myselfDataset(valid_x, valid_y)
  train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
  valid_loader = DataLoader(dataset=validset, batch_size=batch_size, shuffle=True)
  return train_loader, valid_loader

#data_length
length = len(x)
print(length)

dataset = myselfDataset(x,y)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
#train_loader, valid_loader = get_dataloader(batch_size)
#print(train_loader)
#print(valid_loader)

27500


In [15]:
#initialization
#batch_size = 20
num_epochs = 1500
input_size = 2
hidden_size = 2
layer_num = 5


#LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layer_num):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.layer_num = layer_num

        self.lstm = nn.LSTM(input_size, hidden_size, layer_num, batch_first=True, dropout=0.2)
        #self.lstm1 = nn.LSTMCell(self.input_size, self.hidden_size)
        #self.lstm2 = nn.LSTMCell(self.hidden_size, self.hidden_size)
        #self.lstm3 = nn.LSTMCell(self.hidden_size, self.hidden_size)
        self.fc1 = nn.Linear(hidden_size, input_size)
        #self.fc2 = nn.Linear(108, 2)

        #self.relu = nn.ReLU()

    #forward
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_num, x.size(0), self.hidden_size).requires_grad_().to(device)
        # Initialize cell state
        c0 = torch.zeros(self.layer_num, x.size(0), self.hidden_size).requires_grad_().to(device)
        #h1 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)
        #c1 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)
        #h2 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)
        #c2 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)
        #h3 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)
        #c3 = torch.zero(x.size(0), self.hidden_size, dtype=torch.float32).to(device)

        # weights initialization
	      #torch.nn.init.xavier_normal_(h1)
	      #torch.nn.init.xavier_normal_(c1)
	      #torch.nn.init.xavier_normal_(h2)
	      #torch.nn.init.xavier_normal_(c2)
        #torch.nn.init.xavier_normal_(h3)
        #torch.nn.init.xavier_normal_(c3)

        out, _ = self.lstm(x, (h0.detach(), c0.detach()))  
        out = self.fc1(out[:, -1, :])
        #out = self.fc2(out)
        return out

#loss and optimizer
lstm = LSTM(input_size, hidden_size, layer_num).to(device)
learning_rate = 0.05
loss = nn.MSELoss().to(device)
optimizer = optim.SGD(lstm.parameters(), lr=learning_rate, momentum = 0.9)

#lambda1 = lambda epoch: 0.95
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1) 

####hidden_state and cell_state initialize randomly
##h0 = torch.randn(layer_num, batch_size , hidden_size).to(device)
##c0 = torch.randn(layer_num, batch_size , hidden_size).to(device)

#training loop
FADarray = []
MADarray = []

for epoch in range(num_epochs):
      MADdist = 0
      for i, (inputs, labels) in enumerate(train_loader):
        #print(inputs)
        #inputs = inputs.reshape(batch_size, 60, input_size).to(device)
        #print(inputs.shape)

        #forward
        y_pred = lstm(inputs.to(device))
        #print(y_pred)
        l = loss(y_pred, labels.to(device))

        #backward
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    
        MADdistx = 0
        for x in range(len(y_pred)):
            y = y_pred[x].cpu().detach().numpy()
            MADa = np.array([y[0], y[1]])
            MADb = np.array([labels[x][0], labels[x][1]])
            MADdistx += np.sqrt(np.sum(np.square(MADa - MADb)))
        MADdist += MADdistx/len(y_pred)
    
        #if (i+1) % 200 == 0:
          #print(f"epoch = {epoch+1}, step = {i+1} , y's prediction = {y_pred[0].tolist()}, loss = {l.item():.4f}")
      #FADdisty = 0
      #for y in range(len(y_pred)):
          #FADa = y_pred[y].cpu().detach().numpy()
          #FADb = np.array(labels[y])
          #FADdisty += np.sqrt(np.sum(np.square(FADa - FADb)))
      #FADdist = FADdisty/len(y_pred)
      MADdist = MADdist/len(train_loader)
      print(f"epoch = {epoch+1}, Mad = {MADdist}, y's prediction ={y_pred[0].tolist()}, label = {labels[0]}")
      #FADarray.append(FADdist)
      MADarray.append(MADdist)
      scheduler.step()


epoch = 1, Mad = 3903.9517063497133, y's prediction =[4563.48046875, 4890.02880859375], label = tensor([1549.2596, 1015.8469])
epoch = 2, Mad = 3826.698990731775, y's prediction =[4597.02587890625, 4937.5576171875], label = tensor([ 814.6652, 7572.3164])
epoch = 3, Mad = 3825.7538455354425, y's prediction =[4523.8603515625, 4956.064453125], label = tensor([4187.0337, 7237.5820])
epoch = 4, Mad = 3826.102413425698, y's prediction =[4580.146484375, 4925.59619140625], label = tensor([6154.5757, 8174.9478])
epoch = 5, Mad = 3826.636429449795, y's prediction =[4489.267578125, 4952.62744140625], label = tensor([1647.8033,  212.8964])


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt 
epocharray = []
epocharray = [i for i in range(num_epochs)]

#plt.plot(epocharray,FADarray, color="blue")
plt.plot(epocharray,MADarray, color="red")
plt.title("LSTM_100node",fontsize=15)
plt.xlabel("Epoch",fontsize=13)
plt.ylabel("Meters",fontsize=13)
plt.savefig('LSTM_100node.png')
plt.show()